In [ ]:
# !pip install -q selenium

In [ ]:
# !pip install sqlalchemy

In [ ]:
# !pip install beautifulsoup4

In [ ]:
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install -q selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
)

def setup_driver():
  chrome_options = Options()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  return webdriver.Chrome(options=chrome_options)

In [ ]:
driver=setup_driver()
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import psycopg2
from sqlalchemy import create_engine, Column, Integer, Text, BigInteger, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import ARRAY



In [ ]:
driver.get("https://www.hmtwatches.in/womens")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time

def load_all_content(driver):
    page_number = 1
    while True:
        try:
            # Wait for the button to be present and visible
            read_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "loadMoreBtnData"))
            )

            # Scroll the button into view
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", read_more_button)

            # Wait a short moment for the scroll to complete
            time.sleep(2)

            # Try regular click first
            try:
                read_more_button.click()
            except ElementClickInterceptedException:
                # If regular click fails, try JavaScript click
                driver.execute_script("arguments[0].click();", read_more_button)

            print(f"Successfully loaded page {page_number}")
            page_number += 1

            # Wait for the new content to load
            time.sleep(5)

        except TimeoutException:
            print("No more 'Read More' button found. All content loaded.")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            break

# Usage
driver = setup_driver()
driver.get("https://www.hmtwatches.in/womens")

# Load all content
load_all_content(driver)

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, "bc_p_item")

len(elements)

In [ ]:
elements = driver.find_elements(By.CLASS_NAME, "bc_p_item")
count=0
link_list=[]
for element in elements:
  try:
    anchor_tag = element.find_element(By.TAG_NAME, "a")
    link = anchor_tag.get_attribute("href")
    link_list.append(link)

  except NoSuchElementException:
    print("Anchor tag not found in this element.")

In [ ]:
len(link_list)

97

In [ ]:
link1=link_list[0]

In [ ]:
more_info = {}
element = soup.find(id="lD")
if element:
    details = element.text.strip().split('\n')
    for detail in details:
        if ":" in detail:
            key, value = detail.split(":", 1)
            more_info[key.strip()] = value.strip()

In [ ]:
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine, Column, Integer, BigInteger, Text, JSON, ARRAY
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.engine.url import URL

# PostgreSQL database configuration
 = {
    'drivername': '
    'username': '
    'password': '
    'host': '
    'port': 
    'database': 
}

def create_db_connection():
    try:
        # Create URL object using SQLAlchemy's URL.create
        url = URL.create(**)

        # Create engine with the URL object
        engine = create_engine(url)

        # Test the connection
        with engine.connect() as connection:
            print("Successfully connected to the database using SQLAlchemy!")
        return engine

    except Exception as e:
        print(f"Error connecting to the database: {e}")
        raise

# Create base class for declarative models
Base = declarative_base()

# Define Product model
class Product(Base):
    __tablename__ = 'product'

    id = Column(Integer, primary_key=True)
    imageurls = Column(ARRAY(Text))
    product_title = Column(Text)
    review = Column(BigInteger)
    product_description = Column(Text)
    price = Column(BigInteger)
    description = Column(Text)
    product_detail_dict = Column(JSON)
    more_info = Column(JSON)

# Initialize database connection
try:
    engine = create_db_connection()
    Session = sessionmaker(bind=engine)
    session = Session()
except Exception as e:
    print(f"Failed to establish database connection: {e}")
    exit()

# Process each link in the list
for link in link_list:
    try:
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Extract product details
        product_title = soup.find('h3', class_='product-title').text.strip()
        review_no = soup.find('span', class_='review-no').text.strip()
        review = int(review_no.split(" ")[0])
        product_description = soup.find('p', class_='product-description').text.strip()
        product_price = soup.find('h4', class_='price discountPrice').text.strip()
        price = int(product_price.split(" ")[-1])

        # Extract image URLs
        imageurls = []
        try:
            ul_element = driver.find_element(By.ID, "glasscase")
            li_elements = ul_element.find_elements(By.TAG_NAME, "li")
            for li_element in li_elements:
                try:
                    img_tag = li_element.find_element(By.TAG_NAME, "img")
                    src = img_tag.get_attribute("src")
                    imageurls.append(src)
                except NoSuchElementException:
                    print("Img tag not found in this li element.")
        except NoSuchElementException:
            print("Element with ID 'glasscase' not found.")

        # Extract additional details
        description = soup.find(id="lA").text.strip() if soup.find(id="lA") else ""

        product_detail_dict = {}
        element = soup.find(id="lB")
        if element:
            product_details = element.text.strip()
            each_detail_list = [e.strip() for e in product_details.split("\n\n")]
            for each_detail in each_detail_list:
                each_detail_split = [e.strip() for e in each_detail.split("\n")]
                if len(each_detail_split) == 2:
                    product_detail_dict[each_detail_split[0]] = each_detail_split[1]

        more_info = {}
        element = soup.find(id="lD")
        if element:
            details = element.text.strip().split('\n')
            for detail in details:
                if ":" in detail:
                    key, value = detail.split(":", 1)
                    more_info[key.strip()] = value.strip()

        # Create and insert product record
        sample_product = Product(
            imageurls=imageurls,
            product_title=product_title,
            review=review,
            product_description=product_description,
            description=description,
            price=price,
            product_detail_dict=product_detail_dict,
            more_info=more_info
        )

        try:
            session.add(sample_product)
            session.commit()
            print(f"Successfully inserted product with ID: {sample_product.id}")
        except Exception as e:
            session.rollback()
            print(f"Error inserting product: {e}")

    except TimeoutException:
        print(f"Timeout occurred while loading {link}")
    except Exception as e:
        print(f"An error occurred while processing {link}: {e}")



In [ ]:
# product_title = soup.find('h3', class_='product-title')
# product_title=product_title.text.strip()

In [ ]:
# review_no = soup.find('span', class_='review-no')
# review_no=review_no.text.strip()
# review=int(review_no.split(" ")[0])

In [ ]:
# product_description = soup.find('p', class_='product-description')
# product_description=product_description.text.strip()

In [ ]:
# product_price = soup.find('h4', class_='price discountPrice')
# product_price=product_price.text.strip()
# price=int(product_price.split(" ")[-1])

In [ ]:
# ul_element = driver.find_element(By.ID, "glasscase")
# li_elements = ul_element.find_elements(By.TAG_NAME, "li")
# # Now you have a list of all the <li> elements within the <ul>
# for li_element in li_elements:
#   imageurls = []
#   for li_element in li_elements:
#       try:
#           img_tag = li_element.find_element(By.TAG_NAME, "img")
#           src = img_tag.get_attribute("src")
#           imageurls.append(src)
#       except NoSuchElementException:
#           print("Img tag not found in this li element.")
#   imageurls

In [ ]:

# element = soup.find(id="lA")
# if element:
#     description=element.text.strip()
# else:
#   description=""
#   print("Element with id 'lA' not found.")

'HMT Sangam MGGG 03 PRM White'

In [ ]:
# # prompt: get element with id lA using soup
# element = soup.find(id="lB")
# if element:
#     product_details=element.text.strip()
#     # print(product_details)
#     product_detail_dict={}
#     each_detail_list=product_details.split("\n\n")
#     each_detail_list=[e.strip("\n").strip() for e in each_detail_list]
#     for each_detail in each_detail_list:
#       each_detail_split=each_detail.split("\n")
#       each_detail_split=[e.strip() for e in each_detail_split]
#       if len(each_detail_split)==2:
#         product_detail_dict[each_detail_split[0]]=each_detail_split[1]
#     #Further processing of the element can be done here.
#     #For example, to get its text:
#     #print(element.text)
# else:
#   product_detail_dict={}
#   print("Element with id 'lA' not found.")

In [ ]:
# element = soup.find(id="lD")
# if element:
#     more_info = {}
#     details = element.text.strip().split('\n')
#     for detail in details:
#         detail = detail.strip()
#         if ":" in detail:
#           key, value = detail.split(":", 1)
#           more_info[key.strip()] = value.strip()

# else:
#     more_info = {'': ''}
#     print("Element with id 'lD' not found.")
# more_info

{'More Info': '',
 'Manufactured and Packed By': 'HMT Limited, Auxiliary Business Division, Bengaluru -560013 Customer Care No : 080-28385491',
 'Warranty Summary': '',
 'Covered in Warranty': '1 Year on Movement, Battery and Plating',
 'Not Covered in Warranty': 'Normal Wear and tear, Accidental damage due to negligence, unauthorized service'}

In [ ]:

# # PostgreSQL database configuration
# DB_CONFIG = {
#     'dbname': 
#     'user': 
#     'password': ,
#     'host': '#     'port': 
# }

Successfully connected to the database using SQLAlchemy!


In [ ]:
# from sqlalchemy import create_engine, Column, Integer, Text, BigInteger, JSON
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy.dialects.postgresql import ARRAY

# # Create base class for declarative models
# Base = declarative_base()

# # Define Product model
# class Product(Base):
#     __tablename__ = 'product'

#     id = Column(Integer, primary_key=True)
#     imageurls = Column(ARRAY(Text))
#     product_title = Column(Text)
#     review = Column(BigInteger)
#     product_description = Column(Text)
#     price = Column(BigInteger)
#     description = Column(Text)
#     product_detail_dict = Column(JSON)
#     more_info = Column(JSON)

# Session = sessionmaker(bind=engine)
# session = Session()

# # Create sample product record
# sample_product = Product(
#     imageurls=imageurls,
#     product_title=product_title,
#     review=review,
#     product_description=product_description,
#     description=description,
#     price=price,
#     product_detail_dict=product_detail_dict,
#     more_info=more_info
# )

# # Insert the record
# try:
#     session.add(sample_product)
#     session.commit()
#     print(f"Successfully inserted product with ID: {sample_product.id}")
# except Exception as e:
#     session.rollback()
#     print(f"Error inserting product: {e}")
# finally:
#     session.close()

<ipython-input-247-ec3874571868>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Successfully inserted product with ID: 1


Successfully connected to the database!
All data deleted from the product table and ID index reset.
Database connection closed.


In [ ]:
# prompt: get data of particular id using pyalchemy

# Assuming you have the engine and Product model defined as in the previous code.

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

def get_product_by_id(product_id):
  try:
    product = session.query(Product).filter(Product.id == product_id).first()
    if product:
      return product
    else:
      print(f"Product with ID {product_id} not found.")
      return None
  except Exception as e:
    print(f"Error fetching product: {e}")
    return None
  finally:
    session.close()

# Example usage:
product_id_to_fetch = 1  # Replace with the actual ID you want to retrieve
retrieved_product = get_product_by_id(product_id_to_fetch)

if retrieved_product:
    print(f"Product Title: {retrieved_product.product_title}")
    print(f"Product Price: {retrieved_product.price}")
    # Access other attributes as needed...

Product Title: HMT Sangam MGGG 03 PRM White
Product Price: 2033


In [ ]:
# prompt: ALTER TABLE product
# ADD Gender VARCHAR(20);
# give me code

from sqlalchemy import text

def add_gender_column(engine):
    try:
        with engine.connect() as connection:
            # Use text() to execute raw SQL
            connection.execute(text("ALTER TABLE product ADD Gender VARCHAR(20);"))
            print("Successfully added 'Gender' column to the 'product' table.")
    except Exception as e:
        print(f"Error adding 'Gender' column: {e}")

# Call the function with your database engine
add_gender_column(engine)